In [1]:
import numpy as np 
import pandas as pd 
import os
from keras.models import Sequential
from keras.layers import LSTM, Dense, Reshape
import tensorflow as tf

2024-05-12 09:50:52.587065: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 09:50:52.587257: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 09:50:52.768790: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
datasets = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        datasets[filename.split('.')[0]]=pd.read_csv(os.path.join(dirname, filename),header=None)

In [3]:
train_0 = datasets['CFR_train_0']
train_1 = datasets['CFR_train_1']
train_2 = datasets['CFR_train_2']
train_3 = datasets['CFR_train_3']
train = [train_0, train_1, train_2, train_3]
train = pd.concat(train, axis=0, ignore_index=True)
test = datasets['CFR_test']

In [4]:
train_temp = []
for i in range(0, len(train), 28):
        train_unit = train.iloc[i:i+28].to_numpy()
        train_temp.append(train_unit)
test_temp = []
for i in range(0, len(test), 28):
        test_unit = test.iloc[i:i+28].to_numpy()
        test_temp.append(test_unit)

In [5]:
train = np.array(train_temp)
test = np.array(test_temp)

In [6]:
r = 10
l = 5

In [7]:
X_train = []
y_train = []
X_test = []
y_test = []
for i in range(len(train)-r-l+1):
    X_train.append(train[i:i+r])
    y_train.append(train[i+r:i+r+l])
for i in range(len(test)-r-l+1):
    X_test.append(test[i:i+r])
    y_test.append(test[i+r:i+r+l])


X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)
y_train = np.array(y_train)
y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], -1)

X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)
y_test = np.array(y_test)

In [8]:
y_train = y_train.reshape(-1, l, 28,2)

In [9]:
def NMSE(y_true, y_pred):
        y_true_temp = tf.reshape(y_true, [-1, l, 28, 2])
        y_pred_temp = tf.reshape(y_pred, [-1, l, 28, 2])
        error_I = y_true_temp[:,:,:,0]-y_pred_temp[:,:,:,0]
        error_Q = y_true_temp[:,:,:,1]-y_pred_temp[:,:,:,1]
        true_P = tf.reduce_sum(tf.square(y_true_temp[:,:,:,0])) + tf.reduce_sum(tf.square(y_pred_temp[:,:,:,1]))
        NMSE = 10 * tf.math.log((tf.reduce_sum(tf.square(error_I)) + tf.reduce_sum(tf.square(error_Q))) / true_P) / tf.math.log(10.0)
        return NMSE

In [10]:
model = Sequential()
model.add(LSTM(50, activation='relu',input_shape=(r,56)))
model.add(Dense(l * 56))
model.add(Reshape((l, 28 ,2)))
model.compile(optimizer='adam', loss=NMSE)
model.fit(X_train, y_train, epochs=50, verbose=1)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - loss: 0.1818
Epoch 2/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - loss: -0.2668
Epoch 3/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.2991
Epoch 4/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.3314
Epoch 5/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 44s 6ms/step - loss: -0.2758
Epoch 6/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.3386
Epoch 7/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - loss: -0.3962
Epoch 8/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.4244
Epoch 9/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.4882
Epoch 10/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.4921
Epoch 11/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: -0.4776
Epoch 12/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 46s 6ms/step - loss: -0.5037
Epoch 13/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 47s 6ms/step - loss: -0.4846
Epoch 14/50
8072/8072 ━━━━━━━━━━━━━━━━━━━━ 45s 6ms/step - loss: -0.5373
Ep

In [11]:
y_pred = model.predict(X_test)
NMSE = 10 * np.log10((np.sum((y_test[:,:,:,0]-y_pred[:,:,:,0])**2) + np.sum((y_test[:,:,:,1]-y_pred[:,:,:,1])**2)) /
                     (np.sum(y_test[:,:,:,0]**2) + np.sum(y_test[:,:,:,1]**2)))
print("记忆深度：",'(',l,',',r,')',"归一化均方误差：",NMSE)

2106/2106 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step
记忆深度： ( 5 , 10 ) 归一化均方误差： 1.9562688006569997
